<a href="https://colab.research.google.com/github/DhruvDarda/Persuasive-Text-Generation-in-Fashion/blob/main/Transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
from torchtext.legacy import data
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import Tensor 
import pandas as pd
from sklearn.model_selection import train_test_split
from rouge import Rouge

In [ ]:
#%pip install numpy -U
import pandas as pd
from torchtext.data.metrics import bleu_score
from torch.utils.tensorboard import SummaryWriter
from google.colab import drive
drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/'NLP PROJECT'

writer = SummaryWriter('Transformer/largerwithdecoder')

Mounted at /content/gdrive/
/content/gdrive/My Drive/NLP PROJECT


In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download("punkt")

inputs = open('inputs_final1.txt').read().split('\n')
outputs = open('outputs_final.txt').read().split('\n')
inputs = inputs[:min(len(inputs), len(outputs))]
outputs = outputs[:min(len(inputs), len(outputs))]

print(len(set(outputs)), len(outputs))

tokenizer = nltk.RegexpTokenizer(r"\w+")
for i in range(len(inputs)):
  new_words = tokenizer.tokenize(inputs[i])
  inputs[i] = ' '.join(new_words)

raw_data = {'inputs':[line for line in inputs],
            'outputs': [line for line in outputs]}
df = pd.DataFrame(raw_data, columns = ['inputs', 'outputs'])
print(df.head())

print(max(inputs))

train, test = train_test_split(df, test_size = 0.2)

train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
8643 35273
                                              inputs                                            outputs
0  Women Navy Blue amp Rust Orange Printed Straig...  This fashionable kurta from anayna will take y...
1  Green amp Gold Toned Silk Blend Embroidered Sa...  This fashionable kurta from anayna will take y...
2  Women Green amp Pink Embroidered Straight Kurt...  Comfortable and stylish, this kurta from Vbuyz...
3  Dusty Pink Embroidered Semi Stitched Lehenga a...  Comfortable and stylish, this kurta from Vbuyz...
4  Women Pink Ethnic Motifs Yoke Design Regular G...  Comfortable and stylish, this kurta from Vbuyz...
women Pink amp White Ethnic Motifs Hand Printed Bell Sleeves Cotton A Line Kurta Maaesa Pink white hand ethnic motifs hand printed kurta Three quarter bell sleeves Round neck with mock button placket closure Calf length Straight hem A line shape with reg

NameError: ignored

In [ ]:
SOS_WORD = '<sos>'
EOS_WORD = '<eos>'
BLANK_WORD = "<blank>"
MAX_LEN = 70
SRC = data.Field(sequential = True, lower = False, pad_token= BLANK_WORD,fix_length=MAX_LEN,init_token = SOS_WORD, eos_token = EOS_WORD)
TRG = data.Field(sequential = True, lower = False, pad_token= BLANK_WORD,fix_length=MAX_LEN,init_token = SOS_WORD, eos_token = EOS_WORD)
fields = (("SPEC", SRC), ("PERS", TRG))

train_data, test_data = data.TabularDataset.splits(path='./', train='train.csv', test= 'test.csv',\
                                            format='csv', fields=fields, skip_header=True)

In [ ]:
SRC.build_vocab(train_data, test_data)
TRG.build_vocab(train_data, test_data)

In [ ]:
class Transformer(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/p/c80afbc9ffb1/
    """
    # Constructor
    def __init__(
        self,
        num_tokens_inp,
        num_tokens_out,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "Transformer"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=70
        )
        self.embedding1 = nn.Embedding(num_tokens_inp, dim_model)
        self.embedding2 = nn.Embedding(num_tokens_out, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
        )
        self.out = nn.Linear(dim_model, num_tokens_out)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        src_pad_mask = self.make_len_mask(src)
        tgt_pad_mask = self.make_len_mask(tgt)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
        src = self.embedding1(src) * math.sqrt(self.dim_model)
        tgt = self.embedding2(tgt) * math.sqrt(self.dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        
        # We could use the parameter batch_first=True, but our KDL version doesn't support it yet, so we permute
        # to obtain size (sequence length, batch_size, dim_model),
        src = src.permute(1,0,2)
        tgt = tgt.permute(1,0,2)

        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        out = self.out(transformer_out)
        
        return out #(batch_size,sequence_length,French_vocab)
      
    def make_len_mask(self, inp):
        #print(inp.reshape(1,2))
        return (inp == 0)#.transpose(0, 1)

    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask
    
    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]
        return (matrix == pad_token)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device="cpu"

In [ ]:
BATCH_SIZE = 8

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort_key = lambda x: len(x.SPEC),
    sort_within_batch=True)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

model = Transformer(
    num_tokens_inp=INPUT_DIM,num_tokens_out=OUTPUT_DIM, dim_model=200, num_heads=4, num_encoder_layers=3, num_decoder_layers=3, dropout_p=0.1
).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(PATH, model, optimizer):
    print("=> Loading checkpoint")
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    epoch = checkpoint['epoch']
    return epoch, model, optimizer

In [ ]:
def train_loop(model, opt, loss_fn, dataloader, epoch):
    model.train()
    total_loss = 0
    path = %pwd
    PATH = os.path.join(path, 'my_checkpoint.pth.tar')
    #epoch, model, opt = load_checkpoint(PATH, model, opt)
    for batch in tqdm(dataloader):
        X, y = batch.SPEC.T, batch.PERS.T

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]
        
        # Get mask to mask out the next words
        sequence_length = y_input.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)

        # Standard training except we pass in y_input and tgt_mask
        pred = model(X, y_input, tgt_mask)

        # Permute pred to have batch size first again
        pred = pred.permute(1, 2, 0)      
        loss = loss_fn(pred, y_expected)

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()

    checkpoint = {"state_dict" : model.state_dict(), "optimizer" : opt.state_dict(), "epoch" : epoch}
    save_checkpoint(checkpoint)
        
    return total_loss / len(dataloader)

In [ ]:
def get_inp_sen_tensor():
  sentence = "Keep your look simple yet stylish by wearing this yellow slim fit long kurta from Fabindia. Tailored to sartorial perfection from premium quality fabric, it assures a soft and soothing touch against the skin. The mandarin collar augments its design. Exuding full sleeves, it is sure to fetch you compliments from all."

  tokens = tokenizer.tokenize(sentence) #[token for token in sentence]
    # Add <SOS> and <EOS> in beginning and end respectively
  #print(tokens)
  tokens.insert(0, SRC.init_token)
  tokens.append(SRC.eos_token)

    # Go through each german token and convert to an index
  text_to_indices = [SRC.vocab.stoi[token] for token in tokens]
  #print(text_to_indices)

    # Convert to Tensor
  sentence_tensor = torch.LongTensor(text_to_indices).to(device).reshape(1, -1)
  #sentence_tensor = batchify(sentence_tensor, 8)

  #print(sentence_tensor.shape)

  return sentence_tensor
    

In [ ]:
def predict(model, input_sequence, max_length=70, SOS_token=2, EOS_token=3):
    model.eval()
    #itos = SRC.vocab.itos()
    
    y_input = torch.tensor([[SOS_token]], dtype=torch.long, device=device)
    # num_tokens = len(input_sequence[0])
    output_sen = []

    for _ in range(max_length):
        # Get source mask
        tgt_mask = model.get_tgt_mask(y_input.size(1)).to(device)
        
        pred = model(input_sequence, y_input, tgt_mask)
        
        next_item = pred.topk(1)[1].view(-1)[-1].item() # num with highest probability
        next_item = torch.tensor([[next_item]], device=device)
        #print(SRC.vocab.itos[next_item.cpu().detach().numpy()[0][0]])
        output_sen.append(TRG.vocab.itos[next_item])

        # Concatenate previous input with predicted best word
        y_input = torch.cat((y_input, next_item), dim=1)
        #print(y_input)

        # Stop if model predicts end of sentence
        if next_item.view(-1).item() == EOS_token:
            break

    return ' '.join(output_sen) #y_input.view(-1).tolist()
    #vocab.itos(idx)

In [ ]:
def validation_loop(model, loss_fn, dataloader):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            X, y = batch.SPEC.T, batch.PERS.T
            # X, y = torch.tensor(X, dtype=torch.long, device=device), torch.tensor(y, dtype=torch.long, device=device)

            # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
            y_input = y[:,:-1]
            y_expected = y[:,1:]
            
            # Get mask to mask out the next words
            sequence_length = y_input.size(1)
            tgt_mask = model.get_tgt_mask(sequence_length).to(device)

            # Standard training except we pass in y_input and src_mask
            pred = model(X, y_input, tgt_mask)

            # Permute pred to have batch size first again
            pred = pred.permute(1, 2, 0)      
            loss = loss_fn(pred, y_expected)
            total_loss += loss.detach().item()

    print(predict(model, get_inp_sen_tensor()))

    return total_loss / len(dataloader)

In [ ]:
def fit(model, opt, loss_fn, train_dataloader, val_dataloader, epochs):
    # Used for plotting later on
    train_loss_list, validation_loss_list = [], []
    path = %pwd
    PATH = os.path.join(path, 'my_checkpoint.pth.tar')
    s_epoch = 0
    #s_epoch, model, opt = load_checkpoint(PATH, model, opt)

    print("Training and validating model")
    for epoch in range(s_epoch+1, epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        
        train_loss = train_loop(model, opt, loss_fn, train_dataloader, epoch)
        writer.add_scalar("Training loss", train_loss, global_step=epoch)
        train_loss_list += [train_loss]
        
        validation_loss = validation_loop(model, loss_fn, val_dataloader)
        writer.add_scalar("Validation loss", validation_loss, global_step=epoch)
        validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
        print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list, validation_loss_list

epochs = 70
train_loss_list, validation_loss_list = fit(model, opt, loss_fn, train_iterator, test_iterator, epochs)

Training and validating model
------------------------- Epoch 2 -------------------------


100%|██████████| 1246/1246 [00:59<00:00, 21.03it/s]


=> Saving checkpoint


100%|██████████| 312/312 [00:05<00:00, 55.58it/s]


This elegant kurta from Biba will help you maintain an elegant look all year long. you maintain an elegant look all year long. you maintain an elegant look all year long. you maintain an elegant look all year long. you maintain an elegant look all year long. you maintain an elegant look all year long. flats. <eos>
Training loss: 1.5815
Validation loss: 1.3359

------------------------- Epoch 3 -------------------------


...

This breathable and stylish kurta from Biba is a must-have item for any wardrobe. Sport this pink piece with studded flats and minimal jewellery for a simple yet stunning evening ensemble. <eos>
Training loss: 0.6394
Validation loss: 0.5350

------------------------- Epoch 20 -------------------------


100%|██████████| 1246/1246 [00:59<00:00, 21.02it/s]


=> Saving checkpoint


 30%|███       | 94/312 [00:01<00:03, 54.72it/s]


KeyboardInterrupt: ignored

In [ ]:
plt.plot(train_loss_list, label = "Train loss")
plt.plot(validation_loss_list, label = "Validation loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.show()

In [ ]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir="Transformer"

In [ ]:
path = %pwd
PATH = os.path.join(path, 'my_checkpoint.pth.tar')
epoch, model, opt = load_checkpoint(PATH, model, opt)

print(SRC.vocab.stoi['<sos>'])
print(TRG.vocab.stoi['<sos>'])
print(SRC.vocab.stoi['<eos>'])
print(TRG.vocab.stoi['<eos>'])
print(TRG.vocab)

In [ ]:
for i in test_iterator:
  example = i
  #print(example)
  X = example.SPEC
  print(X.shape)
  y = example.PERS
  #print(y)
  break

In [ ]:
print(predict(model,X[:,0].reshape(1,-1)))#.apply(TRG.vocab.itos()))
lol = [] #map(lambda x: TRG.vocab.itos(x), list(y[:,0].cpu().numpy()))
next_item = y.topk(1)[1].view(-1)[-1].item() # num with highest probability
next_item = torch.tensor([[next_item]], device=device)
lol.append(TRG.vocab.itos[next_item])
print(lol)

In [ ]:
predict(model, get_inp_sen_tensor())

In [ ]:
get_inp_sen_tensor()